## 5 EMA Stratergy Implementation
##### stratergy explained : <a href = "https://www.youtube.com/watch?v=uu3zGIXaG8g&ab_channel=PushkarRajThakur%3ABusinessCoach">link</a>
<hr>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import talib as ta
import yfinance as yf
import datetime

In [2]:
data = yf.download(tickers='^NSEI', period='7d', interval='5m')
data.to_csv("test.csv")

[*********************100%***********************]  1 of 1 completed


In [3]:
data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/test.csv')
# data = pd.read_csv('/Users/tejasmadhukar/Documents/Vscode/Data_s/Probability project/Data/NIFTY 50_5min.csv')
# data = data[data['Datetime'].dt.year==2021]
data['Datetime'] = pd.to_datetime(data['Datetime'])
data['5EMA'] = ta.EMA(data['Close'],5)

In [5]:
# data = data.iloc[75:]
#data = data[data['Datetime'].dt.month==1]
data

,Datetime,Open,High,Low,Close,Adj Close,Volume,5EMA
0,2023-01-31 09:15:00,17731.449219,17731.449219,17611.250000,17612.800781,17612.800781,0,NaN
1,2023-01-31 09:20:00,17614.800781,17649.900391,17598.550781,17621.099609,17621.099609,0,NaN
2,2023-01-31 09:25:00,17618.900391,17632.849609,17592.199219,17614.099609,17614.099609,0,NaN
3,2023-01-31 09:30:00,17615.750000,17616.150391,17572.099609,17573.199219,17573.199219,0,NaN
4,2023-01-31 09:35:00,17572.400391,17616.150391,17572.400391,17597.449219,17597.449219,0,17603.729687
...,...,...,...,...,...,...,...,...
467,2023-02-08 10:40:00,17831.300781,17831.300781,17810.449219,17812.349609,17812.349609,0,17831.351002
468,2023-02-08 10:45:00,17812.050781,17822.099609,17799.949219,17801.550781,17801.550781,0,17821.417595
469,2023-02-08 10:50:00,17801.849609,17821.150391,17801.349609,17820.849609,17820.849609,0,17821.228267
470,2023-02-08 10:55:00,17820.500000,17831.250000,17819.900391,17825.900391,17825.900391,0,17822.785641


In [6]:
def enter(data,j,sl,entry,target,arr,ana,count,sl_m,w_m):
    k = j+2
    while(k<len(data['Close'])):
        
        if data['Datetime'].dt.date.values[k] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[k]] = 0
            
        if data['Datetime'].dt.date.values[k] not in w_m:
            w_m[data['Datetime'].dt.date.values[k]] = 0
            
        if(data['Datetime'].dt.day.values[k]>data['Datetime'].dt.day.values[j]):
            change = (entry-(data['High'].values[k-1]))/entry
            arr.append(change)
            ch_b = change>0
            if(change<0):
                sl_m[data['Datetime'].dt.date.values[k]] += 1
            else:
                w_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{7}, entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b,data['Datetime'].dt.date.values[k])
            return k
        
        if(data['High'].values[k]>sl):
            change = (entry-sl)/entry                                                                                  
            arr.append(change)
            sl_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "-ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time:{4} , exit-time:{5}".format(sl,entry,sl,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k 
        
        if(data['Low'].values[k]<target):
            change = (entry-target)/entry                                                                                                                   
            arr.append(change)
            w_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "+ve sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{6}, entry-time:{4} , exit-time:{5}".format(sl,entry,target,target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],data['Datetime'].dt.date.values[k])
            return k
        
        # if(data['Close'].values[k]<target):
        #     if(data['Close'].values[k+1]<target):
        #         target = entry - (4*(sl-entry))
                
    
        if(data['Low'].values[k]-data['5EMA'].values[k]>0.4):
            change = (entry-data['Close'].values[k])/entry
            arr.append(change)
            ch_b = change>0
            if(change<0):
                sl_m[data['Datetime'].dt.date.values[k]] += 1
            else:
                w_m[data['Datetime'].dt.date.values[k]] += 1
            ana[count] = "{6} sl:{0} , entry:{1} , exit:{2} , target:{3} , Date:{7}, entry-time{4} , exit-time:{5}".format(sl,entry,data['Close'].values[k],target,data['Datetime'].dt.time.values[j+1],data['Datetime'].dt.time.values[k],ch_b,data['Datetime'].dt.date.values[k])
            return k-2
        
        k += 1

In [7]:
def trade(data,i,arr,ana,count,sl_m,w_m):
    sl = 0.0
    sl_h = 0.0
    entry = 0.0
    target = 0.0
    j = i+1
    while(j<len(data['Close'])-1):
        if(data['High'].values[j]>data['5EMA'].values[j] and data['Low'].values[j]<data['5EMA'].values[j]):
            sl = data['High'].values[j-1]
            entry = data['Open'].values[j+1] + ((data['Close'].values[j+1]-data['Open'].values[j+1])/2)
            target = entry - (3*(sl-entry))
            if(entry>sl):
                return j 
            return enter(data,j,sl,entry,target,arr,ana,count,sl_m,w_m)
        j += 1

In [8]:
def EMA(arr,data,ana,sl_m,w_m):
    i = 0
    count = 0
    while(i<len(data['Close'])-1):
        
        if data['Datetime'].dt.date.values[i] not in sl_m:
            sl_m[data['Datetime'].dt.date.values[i]] = 0
            
        if data['Datetime'].dt.date.values[i] not in w_m:
            w_m[data['Datetime'].dt.date.values[i]] = 0
            
        if(sl_m[data['Datetime'].dt.date.values[i]]>1):
            i += 1
            continue
            
        if(data['Low'].values[i]>data['5EMA'].values[i]):
            count += 1
            a = trade(data,i,arr,ana,count,sl_m,w_m)
            try:
                b = a-i+1
                i += b
                continue
            except:
                c = 1+1
        i += 1

In [9]:
arr = []
ana = {}
sl_m = {}
w_m = {}
EMA(arr,data,ana,sl_m,w_m)                

In [10]:
amount  = 100000
ini = amount
for i in arr:
    amount = (1+i)*amount
print("Initial: {}".format(ini))
print("P&L before brokerage: {}".format(amount-ini))
print("Percentage change: {}".format(((amount-ini)/ini)*100))

Initial: 100000
P&L before brokerage: 684.4971460571687
Percentage change: 0.6844971460571687


In [11]:
sl_m

{datetime.date(2023, 1, 31): 2,
 datetime.date(2023, 2, 1): 2,
 datetime.date(2023, 2, 2): 2,
 datetime.date(2023, 2, 3): 2,
 datetime.date(2023, 2, 6): 2,
 datetime.date(2023, 2, 7): 2,
 datetime.date(2023, 2, 8): 1}

In [12]:
type(ana)

dict

In [13]:
ana

{1: '-ve sl:17607.94921875 , entry:17602.5498046875 , exit:17607.94921875 , target:17586.3515625 , Date:2023-01-31, entry-time:10:40:00 , exit-time:10:45:00',
 2: '-ve sl:17615.599609375 , entry:17591.875 , exit:17615.599609375 , target:17520.701171875 , Date:2023-01-31, entry-time:11:00:00 , exit-time:11:25:00',
 3: 'False sl:17798.19921875 , entry:17749.275390625 , exit:17781.900390625 , target:17602.50390625 , Date:2023-02-01, entry-time09:35:00 , exit-time:10:05:00',
 6: '-ve sl:17814.25 , entry:17801.400390625 , exit:17814.25 , target:17762.8515625 , Date:2023-02-01, entry-time:10:40:00 , exit-time:10:45:00',
 7: '+ve sl:17650.30078125 , entry:17616.4248046875 , exit:17514.796875 , target:17514.796875 , Date:2023-02-02, entry-time:09:50:00 , exit-time:10:25:00',
 8: '-ve sl:17624.25 , entry:17610.75 , exit:17624.25 , target:17570.25 , Date:2023-02-02, entry-time:11:20:00 , exit-time:11:50:00',
 9: '+ve sl:17634.94921875 , entry:17610.9501953125 , exit:17538.953125 , target:17538.9

In [62]:
a = []
for key,item in ana.items():
    temp = [i.strip() for i in item.split(',')]
    a.append(temp)

In [63]:
for i in a:
    if i[4]=='Date:2023-01-24':
        print(i)

['+ve sl:18197.5', 'entry:18180.1494140625', 'exit:18128.09765625', 'target:18128.09765625', 'Date:2023-01-24', 'entry-time:09:50:00', 'exit-time:10:10:00']
['-ve sl:18171.25', 'entry:18159.875', 'exit:18171.25', 'target:18125.75', 'Date:2023-01-24', 'entry-time:10:55:00', 'exit-time:11:05:00']
['+ve sl:18143.55078125', 'entry:18133.150390625', 'exit:18101.94921875', 'target:18101.94921875', 'Date:2023-01-24', 'entry-time:13:10:00', 'exit-time:13:40:00']
['-ve sl:18113.80078125', 'entry:18109.6005859375', 'exit:18113.80078125', 'target:18097.0', 'Date:2023-01-24', 'entry-time:14:15:00', 'exit-time:14:20:00']


<hr>
<H1>Options Backtest from here</h1>
<hr>